# Counting and Visualizing Document Totals by Year and Source within the Project

### INFO

__author__    = 'Lindsay Thomas'  
__copyright__ = 'copyright 2019, The WE1S Project'  
__license__   = 'MIT'  
__version__   = '2.0'</p> 
__email__     = 'lindsaythomas@miami.edu'

This notebook counts the number of documents per unique source per year in the project. It offers two different methods of counting, and is organized into numbered sections. Results may vary depending on which counting method you use. If you are using dfr-browser's metadata file to do your counting, you must have already produced a dfr-browser for your model. You can configure this setting in section 1. 

The notebook also includes a few options for saving and visualizing these count totals.

You must run every cell under the "Settings" section of this notebook first. Every time you return to this notebook, you must run these cells first.

## Settings

In [ ]:
# Python imports
import os
from pathlib import Path
from IPython.display import display, HTML

# Define paths
current_dir     = %pwd
current_pathobj = Path(current_dir)
project_dir     = str(current_pathobj.parent.parent)
current_reldir  = current_dir.split("/write/")[1]
data_dir        = project_dir + '/project_data'
json_dir        = project_dir + '/project_data/json'
json_test       = project_dir + '/project_data/json_test'
model_dir       = data_dir + '/models'
md_file         = project_dir + '/project_data/metadata/metadata-dfrb.csv'

# Import scripts
%run scripts/count_docs.py
%run scripts/source.py

# Display the project directory
display(HTML('<p><strong>Project Directory:</strong> ' + project_dir + '</p>'))

## 1. Load an Existing Dataframe 

If you have already created and saved a dataframe using this notebook, you can load it for use in this notebook using the cells below. After loading your dataframe, you can skip to section 3. If you need to create a counts dataframe, continue to section 2. 

In [ ]:
# define path of saved dataframe
csv_file = ''

In [ ]:
df = pd.read_csv(csv_file)

## 2. Count Documents by Source and by Publication Year

Make a dataframe with document totals for each unique source by year and download the results. You can obtain source information from EITHER dfr-browser's metadata file (you must have produced a dfr-browser in your project in order to use this method) OR the json documents in your project. Obtaining source information from dfr-browser's metadata file is generally quicker, particularly for large collections of data.

Dates marked as 'unknown' are unknown. To discover the total number of documents listed as having an 'unknown' publication year, scroll down to section 4 of the notebook.

**Make a dataframe of all counts for all sources and all years.** Please select the way you would like to do the counting and specify the names of the `title` and `pub_date` fields in your data. To change how you would like to do the counting, change the value of the `mode` variable to `'dfr-browser'` or `'json'`, depending on which one you want. If you have imported your data using the WE1S import notebook, you do not need to change the values for the `title_field` or `date_field` variables (although if you are using WE1S data, see the next paragraph). Otherwise, change these values to the metadata fields where you would like the code to look for your publication titles and your publication dates.

Please follow the below format exactly for each variable that you change:

mode = `'json'`

After running the below cells, you may skip to section 3 to further visualize your results.

In [ ]:
## mode must be `dfr-browser` or `json`
# mode = 'dfr-browser'
mode = 'json'

# Only change the below values if your title and publication dates are located in fields with 
# different names in your data
title_field = 'source'
date_field = 'pub_date'

In [ ]:
df = source_count_by_year(mode, md_file, json_dir, title_field, date_field)

**View the dataframe for these counts.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
# sort by highest totals, print in descending order
df = df.sort_values('Total', ascending=False)

qgrid_widget = qgrid.show_grid(df, grid_options=grid_options, show_toolbar=False)
qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file named `source_counts_by_year.csv` in this module directory which you can download and save to your computer for further processing and visualization (using Excel, Google sheets, etc).

In [ ]:
# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv('source_counts_by_year.csv', index_label = 'Sources')
display(HTML('<p><strong>CSV file saved.</strong></p>'))

In [ ]:
## Save original dataframe to csv, disregarding any changes you made in qgrid

# df.to_csv('source_counts_by_year.csv', index_label = 'Sources')
# display(HTML('<p><strong>CSV file saved.</strong></p>'))

## 3. Further Explore and Visualize Results

You must have completed section 1 to run any cells in this section.

**Create a dataframe of the top 10 sources for a given year or years.** You can specify the year or years you would like to get counts for using the `years` variable. Enter the year or years you want to count as a list:

`years = ['2014', '2015', '2017']`

You may also get counts for sequential years by specifying a date range as a string, e.g.:

`date_range = '2014-2016'`

The `date_range` variable is set to `None` by default. Please follow the above formats exactly.

In [ ]:
# specify the specific years you want to get counts for
years = ['2014', '2015', '2017']
# specify a date range. Change to None if not using
date_range = None 

df_top10 = df.iloc[:10]

if date_range:
    df_top10 = df_top10.loc[:,date_range.split('-')[0]:date_range.split('-')[1]] 
else:
    df_top10 = df_top10.loc[:,years]

df_top10

**Plot the above dataframe you created using subplots** (i.e., showing a separate plot for each unique source).

In [ ]:
# plot the above dataframe using subplots
# to save the figure for downloading, uncomment the commented out line at the bottom of the cell

%matplotlib inline
import matplotlib.pyplot as plt

df_top10.transpose().plot(kind='bar', figsize=(15,10), subplots=True, title=['','','','','','','','','',''])


**Save the above plot to disk.** Uncomment the lines below to set the plot filename and save it to disk.

In [ ]:
# fig_name = 'top10.png'

# plt.savefig(fig_name)

**Get the total number of documents for a specific source and the total number of documents/year for that source between 2 years.** You can specify the year or years you would like to get counts for using the `years` variable. Enter the year or years you want to count as a list:

`years = ['2014', '2015', '2017']`

You may also get counts for sequential years by specifying a date range as a string, e.g.:

`date_range = '2014-2016'`

The `date_range` variable is set to `None` by default. 

Finally, the value for the `source` variable should be a string that corresponds to the title of the source you want to find. Please note that this title will be taken from whatever metadata field you specified as your `title_field` in the first cell under section 2 above.

Please follow the above formats exactly.

In [ ]:
source = 'the-guardian-london'
specific_years = ['2014','2016']
date_range = '2014-2016' # Use something like '2014-1016' or set to None if not required

try:
    df_total = df.loc[source, 'Total']
    print('Total articles for ' + source + ': ' + str(df_total))
    if date_range:
        df_singlesource = df.loc[source,date_range.split('-')[0]:date_range.split('-')[1]]
    else:
        df_singlesource = df.loc[source,specific_years]
    pd.DataFrame(df_singlesource)
except KeyError:
     display(HTML('<p style="color:#FF0000";>That source title does not exist in dataframe. Check `title_field` variable above.</p>'))


**Plot the above dataframe.** Give your plot a title by assigning the `plot_title` variable.

`plot_title = 'Chicago Daily Herald'`

Please follow that format exactly.

In [ ]:
# plot a single source between 2 years

%matplotlib inline
import matplotlib.pyplot as plt

plot_title = 'Plot Title'

df_singlesource.transpose().plot(kind='bar', color=(0.2, 0.4, 0.6, 0.6), title=plot_title)

**Save the above plot to disk.** Uncomment the lines below to set the plot filename and save it to disk.

In [ ]:
# fig_name = 'singlesource.png'

# plt.savefig(fig_name)

**Count the total number of documents for a given year or years.** You can specify the year or years you would like to get counts for using the `years` variable. Enter the year or years you want to count as a list:

`years = ['2014', '2015', '2017']`

You may also get counts for sequential years by specifying a date range as a string, e.g.:

`date_range = '2014-2016'`

The `date_range` variable is set to `None` by default. Please follow the above formats exactly.

In [ ]:
years = ['2014']
date_range = None # Use something like '2014-1016' or set to None if not required

if date_range:
    year_total = df.loc[:,date_range.split('-')[0]:date_range.split('-')[1]].sum()
else:
    year_total = df.loc[:,years].sum()

pd.DataFrame(year_total, columns=['Total'])

## 4. Count Total Number of Documents without a Publication Date

**Count number of documents without a publication date in dataframe created in this notebook.** You must have run cells under "Settings" and section 1 of this notebook for this to work.

In [ ]:
try:
    count1 = df['1900'].sum()
except KeyError as err:
    count1 = 0
try:
    count2 = df['unknown'].sum()
except KeyError as err:
    count2 = 0
try:
    count3 = df['NaN'].sum()
except KeyError as err:
    count3 = 0

display(HTML('<p>' + str(count1) + ' dates listed as 1900 </p>'))
display(HTML('<p>' + str(count2) + ' dates listed as unknown </p>'))
display(HTML('<p>' + str(count3) + ' dates listed as NaN </p>'))


## 5. Count Documents by Metadata Field

You must have run all of the cells under "Settings" to run the code in this section.

The code below counts the number of documents associated with a specific metadata field in your project. You must provide the field you want to count. The `json_utilities` module includes methods for retrieving lists of all of the metadata fields in your files. You can count the total number of documents with any value for a certain field, or you can count the number of documents that have a certain value within a certain field.

**Count the total number of documents associated with all values for a specific field.** For example, count all of the documents associated with each value in the `tags` field.

In [ ]:
# enter the name of the field you want to count here
field = ''

bad_jsons, no_field, df = docs_by_field(json_dir, field)

Run the below cell to see the number of json documents in your project that failed to load correctly (and so were not read or considered in creating count totals). If this number is large, this will significantly affect the results you can obtain using this notebook.

In [ ]:
len(bad_jsons)

Run the below cell to see the number of json documents in your project that do not contain the field you are trying to count. If this number is large, this will significantly affect the results you can obtain using this notebook.

In [ ]:
len(no_field)

**View the dataframe for these counts.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents with a particular value in your chosen metadata field). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(df, grid_options=grid_options, show_toolbar=False)
qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell.

Either cell will create a csv file named `YOURFIELD_counts.csv` in this module directory which you can download and save to your computer for further processing and visualization (using Excel, Google sheets, etc).

In [ ]:
# save dataframe to csv
changed_df = qgrid_widget.get_changed_df()

csv_file = field + '_counts.csv'

changed_df.to_csv(csv_file, index_label = 'Index')
display(HTML('<p>Dataframe saved as ' + csv_file + '</p>'))

In [ ]:
## Save original dataframe to csv, disregarding any changes you made in qgrid

# csv_file = field + '_counts.csv'

# df.to_csv('source_counts_by_year.csv', index_label = 'Sources')
# display(HTML('<p>Dataframe saved as ' + csv_file + '</p>'))

**Count the number of documents that have a specific value for a specific field.** For example, count the number of documents tagged with 'education/funding/US private college' in your project.

In [ ]:
field = 'tags'
target_value = 'education/funding/US private college'

value_count = specific_value_count(json_dir, field, target_value)

print(str(value_count) + ' documents with ' + target_value + ' value in project')
display(HTML('<p>' + value_count + '  documents with ' + target_value + ' value in project</p>'))

Understand this number in the context of the rest of the project. Run the following code to see the proportion of documents in your project that have your target value for your target field.

In [ ]:
json_dir_files = [file for file in os.listdir(json_dir) if file.endswith('.json')]
json_length = len(json_dir_files)
                
display(HTML('<p>' + str(value_count) + ' documents with ' + target_value + 'value in project</p>'))
display(HTML('<p>' + str(json_length) + ' total documents in project</p>'))
      
proportion = value_count/json_length

display(HTML('<p>' + str(proportion*100) + '% of project jsons have' + target_value + 'as value for ' + field + 'field</p>'))

